<a href="https://colab.research.google.com/github/ljyslyc/comment-analysis/blob/master/Tcommentspider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import json
from lxml import etree
import pandas as pd
import re
import math

##调用getMerchantComment接口获取评论json
def getShopPage(shopID,offSet):
    simulateChromeBrowserData = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host':'sz.meituan.com',
        'Referer':'http://wh.meituan.com/meishi/',
        # 'Cookie': '_lxsdk_cuid=16212a00d8ec8-07cdb6596bad8e-178123e-1fa400-16212a00d8fc8; lat=22.780886; lng=113.906362; client-id=34908f62-ea11-4211-b60a-f62c32288b2e; uuid=9be4f96971ac4c9cab4c.1520730903.1.0.0; webloc_geo=22.527181%2C113.938582%2Cwgs84; ci=30; _lxsdk=16212a1cddfc8-011369480302e7-178123e-1fa400-16212a1cddfc8; __mta=247430459.1520730902128.1520731016684.1520731025187.5; _lxsdk_s=16212a00d8f-c83-6e-376%7C%7C9',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    purl='https://www.meituan.com/meishi/api/poi/getMerchantComment'
    params = {
        'id' : shopID,
        'userId' : 0,
        'offset' : offSet*10,#偏移量，也是页码信息
        'pageSize' : 10,
        'sortType' : 1
    }
    page = requests.get(purl,params=params,headers=simulateChromeBrowserData)
    return page



In [0]:
def dataClean(comment):  #只保留中文
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    commentc = re.sub(pattern, '', comment)
    return commentc

def parse_POI_Shop_Json(shop_json):
    one_comment_data = pd.DataFrame()
    if 'data'in shop_json:
        for eachShopInfo in shop_json.get('data').get('comments'):
            Comment = dataClean(eachShopInfo.get('comment'))
            if not Comment:
                continue
            IsGreet = 1 if eachShopInfo.get('star')>= 30 else 0
            one_comment_data = one_comment_data.append(pd.DataFrame({'Comment':[Comment],'IsGreet':[IsGreet]}))
    return one_comment_data

In [0]:
Comment_datas = pd.DataFrame();
# allCommentNum商家的最大评论数
allCommentNum=150
for i in range(allCommentNum//10):
    page = getShopPage(74779,i)
    #print(page.url)
    # print(page.json())
    Comment_data = parse_POI_Shop_Json(page.json())
    Comment_datas = Comment_datas.append(Comment_data)
print(Comment_datas)
Comment_datas.to_csv('Comment_datas.csv', index=False)

                                              Comment  IsGreet
0                                个人吃加了个羊腿不错的一家西餐厅还会光顾        1
0   我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...        1
0   每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...        1
0   和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...        1
0                                  还不错吧环境优雅菜品种类多味道还可以        1
0                               不好吃都浪费了真不知道秀玉现在怎么这么差了        0
0                                     味道还不错环境比较好就餐很安静        1
0                          环境特别好牛排也特别美味金桔茶特别推荐哦就餐特别推荐        1
0                                        还可以人不多上菜快服务好        1
0               带着弟弟们来吃的位置有点难得找人很多味道不错看来地方不同秀玉的味道差别也大        1
0   邓平小姐姐服务态度很好因为是第一次来这里耐心给我们推荐菜品回答我们的问题这里二楼比较安静喜欢...        1
0                                    味道好吃可以可以男朋友吃的很满意        1
0   环境还不错主要是吃这个衣服上不会有味道饮料也很好喝不是那种用粉冲出来的服务员态度也都很好给我...        1
0                                      味道不错套餐实惠小孩很喜欢吃        1
0                                           味道好很喜欢吃推荐  

In [0]:
#加在停用词库
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding='gbk') as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = "中文停用词库.txt"
stopwords = get_custom_stopwords(stop_words_file)
print(stopwords)

['$', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '_', '“', '”', '、', '。', '《', '》', '一', '一些', '一何', '一切', '一则', '一方面', '一旦', '一来', '一样', '一般', '一转眼', '万一', '上', '上下', '下', '不', '不仅', '不但', '不光', '不单', '不只', '不外乎', '不如', '不妨', '不尽', '不尽然', '不得', '不怕', '不惟', '不成', '不拘', '不料', '不是', '不比', '不然', '不特', '不独', '不管', '不至于', '不若', '不论', '不过', '不问', '与', '与其', '与其说', '与否', '与此同时', '且', '且不说', '且说', '两者', '个', '个别', '临', '为', '为了', '为什么', '为何', '为止', '为此', '为着', '乃', '乃至', '乃至于', '么', '之', '之一', '之所以', '之类', '乌乎', '乎', '乘', '也', '也好', '也罢', '了', '二来', '于', '于是', '于是乎', '云云', '云尔', '些', '亦', '人', '人们', '人家', '什么', '什么样', '今', '介于', '仍', '仍旧', '从', '从此', '从而', '他', '他人', '他们', '以', '以上', '以为', '以便', '以免', '以及', '以故', '以期', '以来', '以至', '以至于', '以致', '们', '任', '任何', '任凭', '似的', '但', '但凡', '但是', '何', '何以', '何况', '何处', '何时', '余外', '作为', '你', '你们', '使', '使得', '例如', '依', '依据', '依照', '便于', '俺', '俺们', '倘', '倘使', '倘或', '倘然', '倘若', '借', '假使', '假如', '假若', '傥然', '像', '儿', '先不先', '光是', '全体', '全部', '

In [0]:
import pkuseg
import numpy as np
Commentdata=np.array(Comment_datas.get('Comment'))
X_data=pd.DataFrame()
Y_data=np.array(Comment_datas.get('IsGreet'))
seg = pkuseg.pkuseg()           # 以默认配置加载模型
for items in Commentdata:
    text = seg.cut(items)  # 进行分词  
    X_data=X_data.append(pd.DataFrame({'cytedCom':text}))



In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())
term_matrix.head()


NameError: ignored

In [0]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

In [0]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())
term_matrix.head()

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score

nb = MultinomialNB()
pipe = make_pipeline(vect, nb)
pipe.steps
cross_val_score(pipe, X_train.cutted_comment, y_train, cv=5, scoring='accuracy').mean()
pipe.fit(X_train.cutted_comment, y_train)
pipe.predict(X_test.cutted_comment)

In [0]:
from sklearn import metrics
y_pred = pipe.predict(X_test.cutted_comment)
metrics.accuracy_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

神经网络模型

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
model = Sequential()
# 搭建模型
model.add(Dense(32, activation='relu', input_shape=(768,)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
X_test_data=X_data[1400:1485]
Y_test_data=Y_data[1400:1485]
X_train = []
y_train = []
X_test = []
y_test = []
nums = np.arange(1400)
# 随机打乱12000个训练数据
np.random.shuffle(nums)
for i in nums:
    X_train.append(X_data[1])
    y_train.append(Y_data[1])

# 随机打乱1600个测试数据
nums_ = np.arange(85)
np.random.shuffle(nums_)  # shuffle改变的是自身的内容
for i in nums_:
    X_test.append(X_test_data [1])
    y_test.append(Y_test_data [1])

# list转Numpy数组
X_train= np.array(X_train)
X_test= np.array(X_test)


In [0]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.train.AdamOptimizer(),
    metrics=['acc']
)
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

In [0]:
import matplotlib.pyplot as plt
history_dict = history.history
epochs = range(1, len(history_dict['acc']) + 1)
# 绘图部分
plt.figure()
plt.plot(epochs, history_dict['acc'], 'b', label='acc')
plt.plot(epochs, history_dict['val_acc'], 'bo', label='val_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
test_loss = model.evaluate(
    X_test,
    y_test,
    batch_size=63,
    verbose=1
)
print(test_loss)